## define histogram function

In [1]:
function hist(array,min,max,Δ)
    #use it like this: hist(rand(1:10,10^6),0.5,10.5,1)
    try
        Integer((max-min)/Δ)
    catch
        println("(max-min)/Δ is not an Integer!")
    end
    y=zeros(Integer((max-min)/Δ))
    error=0
    for el in array
        if min<el<=max
            y[Integer(ceil((el-min)/Δ))]+=1
        else
            if error == 0
                error=1
                println("Range insufficient. Element with value $(el) was found and ignored.")
            end
        end
    end
    x=Array(min:Δ:max)
    return [0.5*(x[1:end-1]+x[2:end]), y]
end

hist (generic function with 1 method)

# load downloaded data + translate into distance array
Downloaded from https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page \
I'm only using the data from "Yellow Taxi".

In [210]:
for month in 1:12
    #load data
    if month<10
        data=readdlm("../../RawData/TaxiNYC/yellow_tripdata_2019-0$(month).csv")
    else
        data=readdlm("../../RawData/TaxiNYC/yellow_tripdata_2019-$(month).csv")
    end
    
    #translate into distances
    distances=zeros(length(data[2:end,3]))
    for jj in 1:length(data[2:end,3])
        str=data[1+jj,3]
        mys=""
        count=0
        for ind in 1:length(str)
            sym=str[ind]
            if sym==','
                count+=1
                continue
            end
            if count==2
                mys*=sym
            elseif count == 3
                break
            end
        end
        distances[jj]=parse(Float64,mys)
    end
    
    #save to variable
    if month<10
        myCode="d0$(month)=distances"
    else
        myCode="d$(month)=distances"
    end
    eval(Meta.parse(myCode))
    
    #proudly annouce progress
    println("month $(month) done.")
end

month 8 done.
month 9 done.
month 10 done.
month 11 done.
month 12 done.


## write distances to files

In [2]:
using DelimitedFiles

In [3]:
for month in 1:12
    if month<10
        myCode="writedlm(\"distances/d0$(month)\",d0$(month))"
    else
        myCode="writedlm(\"distances/d$(month)\",d$(month))"
    end
    eval(Meta.parse(myCode))
end

LoadError: UndefVarError: d01 not defined

## read distances from files

In [4]:
for month in 1:12
    if month<10
        myCode="d0$(month)=readdlm(\"distances/d0$(month)\")"
    else   
        myCode="d$(month)=readdlm(\"distances/d$(month)\")"     
    end
    eval(Meta.parse(myCode))
end

## combine distances into one array

In [5]:
distances=vcat(d01,d02,d03,d04,d05,d06,d07,d08,d09,d10,d11,d12);

## calculate histogram

In [61]:
min=0
max=30
Δ=0.1
his=hist(distances,min,max,Δ)
println("$(round(100*sum(his[2])/length(distances);digits=2))% of data used.")
Z=sum(his[2])*Δ
his[2]./=Z;

Range insufficient. Element with value 0.0 was found and ignored.
99.06% of data used.


## plot histogram

In [11]:
using Plots
plotlyjs()
theme(:juno)

In [47]:
using Statistics

In [70]:
plot(his[1],his[2],xlim=[0,6],marker=:hex,xlab="distance / km",ylab="frequency",title="taxi data from NYC, year 2019")

<!DOCTYPE html>
 
 
 Plots.jl

In [71]:
D=mean(distances)
k=3.1
x=Array(0.01:0.01:20)
y=exp.(-(x./D).^-1).*((x./D).^-k)
y./=sum(y)*0.01
plot!(x,y,xlim=[0,12],xlab="distance / km",ylab="frequency",title="taxi data from NYC, year 2019",xticks=1:50)

<!DOCTYPE html>
 
 
 Plots.jl

In [343]:
savefig("traveldistancesHistogramNYC2019.html")

In [37]:
plot(his[1],-cumsum(his[2]*Δ).+1,marker=:hex,xlim=[0,12],xlab="distance / km",ylab="remaining customers",title="taxi data from NYC, year 2019")

<!DOCTYPE html>
 
 
 Plots.jl

In [38]:
savefig("remainingCustomersNYC2019.html")

In [1]:
#log-log
logd1=log.(10,his[1])
logd2=log.(10,his[2])
plot(logd1,logd2,marker=:hex,xlab="log_10 distance / km",ylab="log_10 frequency",title="taxi data from NYC, year 2019")

LoadError: UndefVarError: his not defined

In [349]:
savefig("traveldistancesHistogramNYC2019_LogLog.html")

## power law exponent - rough estimate

In [345]:
right=300
left=50
(logd2[right]-logd2[left])/(logd1[right]-logd1[left])

-3.1900103156234354